In [1]:
# Imports
import torch.nn as nn
import torch

In [3]:
# Task 0. Generator
class Generator(nn.Module):
    """
    Generator subclass for the generator side of the network
    """
    def __init__(self,
                 input_size,
                 hidden_size,
                 output_size):
        """
        Initialize Generator class

        Args:
            input_size - integer size of the input tensor
            hidden_size - integer size of the hidden layer
            output_size - integer size of the output layer
        """
        super(Generator, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.layers = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, output_size)
        )

        def forward(self, x):
            """
            Forward pass of the Generator network

            Args:
                x - Input tensor of shape (batch_size, input_size)

            Returns:
                Output tensor of shape (batch_size, output_size)
            """
            return self.layers(x)


In [8]:
# 0-main
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
genTester = Generator

G_Test = genTester(1,1,1)

print(G_Test)

Generator(
  (layers): Sequential(
    (0): Linear(in_features=1, out_features=1, bias=True)
    (1): Tanh()
    (2): Linear(in_features=1, out_features=1, bias=True)
    (3): Tanh()
    (4): Linear(in_features=1, out_features=1, bias=True)
  )
)


In [11]:
# Task 1. Discriminator
class Discriminator(nn.Module):
    """
    Discriminator subclass for the discriminator side of the network
    """
    def __init__(self,
                 input_size,
                 hidden_size,
                 output_size):
        """
                Initialize Discriminator class

        Args:
            input_size - integer size of the input tensor
            hidden_size - integer size of the hidden layer
            output_size - integer size of the output layer
        """
        super(Discriminator, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.layers = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.Sigmoid(),
            nn.Linear(hidden_size, hidden_size),
            nn.Sigmoid(),
            nn.Linear(hidden_size, output_size),
            nn.Sigmoid()
        )

        def forward(self, x):
            """
            Forward pass of the Discriminator network

            Args:
                x - Input tensor of shape (batch_size, input_size)

            Returns:
                Output tensor of shape (batch_size, output_size)
            """
            return self.layers(x)

In [13]:
# 1-main
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

disTester = Discriminator

D_Test = disTester(1,1,1)

print(D_Test)

Discriminator(
  (layers): Sequential(
    (0): Linear(in_features=1, out_features=1, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=1, out_features=1, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=1, out_features=1, bias=True)
    (5): Sigmoid()
  )
)


In [121]:
# Task 2. Sample Z
def sample_Z(mu, sigma, sampleType, size=(1, 1)):
    """
    Creates input for the generator and discriminator

    Added size argument to sample_Z function. Per torch.randn and torch.normal
        documentataion, size is a sequence of integers defining the shape of
        the output tensor.

    Args:
        mu -  The mean of the distribution
        sigma - The standard deviation of the distribution
        sampleType - A variable that selects which model to sample for
            The variable should accept a G or D as string values
        size - tuple of ints defining the shape of the output tensor

    Returns:
        a torch.Tensor type for both generator and discriminator if the
            parameters are correct, otherwise return 0
    """
    # For the generator, find a random number from torch.rand
    # Multiply that number by sigma and add mu
    # From the task, input data for the generator should be random
    if sampleType == "G":
        return torch.rand(size=size)
    # For the discriminator, take a number from the normal distribution
    # using torch.normal
    elif sampleType == "D":
        return torch.normal(mean=mu, std=sigma, size=size)
    else:
        return 0


In [122]:
# 2-main
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

samplerTest = sample_Z

print(samplerTest(0,1,"D",(4,8)))

print(samplerTest(0,1,"G",(4,8)))

print(torch.normal(mean=0, std=1, size=(4, 8)))

print(samplerTest(0,1,"F",))

tensor([[ 0.2214,  0.4322, -0.9203,  0.7536,  0.6553,  0.1761, -0.5263, -0.7023],
        [-0.5220, -0.3230,  0.1377, -0.1602,  1.5244, -0.9660,  0.9375, -0.1360],
        [ 1.2241, -0.3380,  1.1565,  0.3569, -0.5476,  0.0958,  1.0942,  0.5019],
        [-0.8832, -0.9448,  0.3071, -0.5883, -0.7596, -0.4670,  0.2064,  2.4383]])
tensor([[ 1.0208,  0.7561, -1.4899,  1.8450,  2.1332, -1.9036,  1.5311, -0.0575],
        [-0.2674, -0.4299, -0.5251,  1.6954, -1.3254,  0.6926,  1.4660,  0.3970],
        [-1.0886,  1.1354,  0.0983,  0.0838, -0.7193,  0.3953,  0.9355,  0.5266],
        [-1.5429, -0.6365, -0.2119, -0.6193, -0.0530,  1.0209, -1.6480, -1.4621]])
tensor([[ 0.3577,  0.3319, -1.7373, -1.0931, -0.7027, -0.4014,  0.5335, -0.9551],
        [ 1.1802,  1.4854, -0.0312,  0.5831, -2.4657,  1.0780, -2.3520,  0.2369],
        [ 1.4022, -0.8666, -0.9594,  0.2335, -1.0007, -0.0715,  2.3987, -0.0598],
        [-2.1644, -0.0546, -0.4469, -0.7767,  0.5255,  0.9842,  1.9959,  1.2033]])
0
